# Auto-sklearn

#### Author's description:

auto-sklearn is an automated machine learning toolkit and a drop-in replacement for a scikit-learn estimator

#### Useful links:

[install link](https://automl.github.io/auto-sklearn/master/installation.html),
[git](https://github.com/automl/auto-sklearn),
[manual](https://automl.github.io/auto-sklearn/master/manual.html),
[parallel instances](https://automl.github.io/auto-sklearn/master/examples/example_parallel_manual_spawning.html),
[parallel runs on one machine](https://automl.github.io/auto-sklearn/master/examples/example_parallel_n_jobs.html),
[cross validation](https://automl.github.io/auto-sklearn/master/examples/example_crossvalidation.html),
[feature types](https://automl.github.io/auto-sklearn/master/examples/example_feature_types.html)

## Install and import

Note that we use the subprocess function instead of the jupyter **!** method of running bash commands. Domino can run these notebooks as [jobs](https://support.dominodatalab.com/hc/en-us/articles/360023696651-Jobs) (batch or scheduled) which turns your ipython notebook into an executable script file! All you have to do is ensure the code can be executed in a .py file.

In [1]:
import subprocess

completed = subprocess.run(['sudo', 'pip', 'install', 'auto-sklearn'], \
                           stdout=subprocess.PIPE,)
print(completed.stdout.decode('utf-8'))

  Created wheel for auto-sklearn: filename=auto_sklearn-0.6.0-cp36-cp36m-linux_x86_64.whl size=4268838 sha256=feb2012b5eb841b880f9f7fe1c65df9520a7538c23edf0ae9a166c9fae984717
  Stored in directory: /tmp/pip-ephem-wheel-cache-w2f8plcb/wheels/1b/9a/78/0dd70afe14ccb1f8d3fb51cedda41b52c4cfc398eeb3efeb60
  Created wheel for liac-arff: filename=liac_arff-2.4.0-py3-none-any.whl size=13335 sha256=de744e702c28e6c8b94cb84919ab2213b208976d91e0fc68261f477a3c973da9
  Stored in directory: /tmp/pip-ephem-wheel-cache-w2f8plcb/wheels/ba/2a/e1/6f7be2e2ea150e2486bff64fd6f0670f4f35f4c8f31c819fb8
  Created wheel for ConfigSpace: filename=ConfigSpace-0.4.12-cp36-cp36m-linux_x86_64.whl size=2949368 sha256=17e390c7c08aae77ea70f03adbf1be3a3eef3372cd7d7f4d04bd340428fe1804
  Stored in directory: /tmp/pip-ephem-wheel-cache-w2f8plcb/wheels/71/b3/ae/cdae924f0b68e679f3558e956dc34cf0e463b80b7c2883b4d5
  Created wheel for pynisher: filename=pynisher-0.5.0-py3-none-any.whl size=4359 sha256=aa1c9e1db619f5947bb53a24b6422

In [2]:
import autosklearn.classification
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import pandas as pd
import numpy as np

In [3]:
#the original notebook was created on 0.5.2
autosklearn.__version__

'0.6.0'

In [4]:
#there can be a lot of warnings in auto-sklearn
#especially if you overwrite existing files
#turning off for demo purposes

import warnings
warnings.filterwarnings("ignore")

## Take a look at the classification function

auto-sklearn is mostly a wrapper around scikit-learn. It was not the intention of the authors to allow user control over details such as the modeling algorithm and typical hyper-parameter choices. Control is several layers deep in the [SMAC](https://automl.github.io/SMAC3/stable/index.html) space and scenario settings. The user can control the time is takes to build the ensemble, the resampling strategy and the parallelization of the work across CPUs on the machine. These will be demonstrated below.

In [5]:
?autosklearn.classification.AutoSklearnClassifier

Init signature:
autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=3600,
    per_run_time_limit=360,
    initial_configurations_via_metalearning=25,
    ensemble_size:int=50,
    ensemble_nbest=50,
    ensemble_memory_limit=1024,
    seed=1,
    ml_memory_limit=3072,
    include_estimators=None,
    exclude_estimators=None,
    include_preprocessors=None,
    exclude_preprocessors=None,
    resampling_strategy='holdout',
    resampling_strategy_arguments=None,
    tmp_folder=None,
    output_folder=None,
    delete_tmp_folder_after_terminate=True,
    delete_output_folder_after_terminate=True,
    shared_mode=False,
    n_jobs:Union[int, NoneType]=None,
    disable_evaluator_output=False,
    get_smac_object_callback=None,
    smac_scenario_args=None,
    logging_config=None,
    metadata_directory=None,
)
Docstring:      This class implements the classification task.
Init docstring:
Parameters
----------
time_left_for_this_task : int, optional (default=3600)

## Heart Disease

#### Load the heart disease dataset

Note that in this cell we are calling **sklearn.model_selection.train_test_split()** twice and creating two sets of heart disease (hd) data for model fitting and testing. One is for the hd data without one hot encoding (ohe) and the other has the ohe columns. 

auto-sklearn accepts a list of categorical features and has several methods for treating categorical data. In this notebook we try both approaches - building ohe columns ourselves and letting auto-sklearn do its thing.

In [6]:
'''
/mnt/data/raw/heart.csv

attribute documentation:
      age: age in years
      sex: sex (1 = male; 0 = female)
      cp: chest pain type
        -- Value 1: typical angina
        -- Value 2: atypical angina
        -- Value 3: non-anginal pain
        -- Value 4: asymptomatic
     trestbps: resting blood pressure (in mm Hg on admission to the 
        hospital)
     chol: serum cholestoral in mg/dl
     fbs: (fasting blood sugar > 120 mg/dl)  (1 = true; 0 = false)
     restecg: resting electrocardiographic results
        -- Value 0: normal
        -- Value 1: having ST-T wave abnormality (T wave inversions and/or ST 
                    elevation or depression of > 0.05 mV)
        -- Value 2: showing probable or definite left ventricular hypertrophy
                    by Estes' criteria
     thalach: maximum heart rate achieved
     exang: exercise induced angina (1 = yes; 0 = no)
     oldpeak = ST depression induced by exercise relative to rest
     slope: the slope of the peak exercise ST segment
        -- Value 1: upsloping
        -- Value 2: flat
        -- Value 3: downsloping
     ca: number of major vessels (0-3) colored by flourosopy
     thal: 
         3 = normal; 
         6 = fixed defect; 
         7 = reversable defect
     target: diagnosis of heart disease (angiographic disease status)
        -- Value 0: < 50% diameter narrowing
        -- Value 1: > 50% diameter narrowing
 '''

#load and clean the data----------------------

#column names
names = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang', \
         'oldpeak','slope','ca','thal','target']

#load data from Domino project directory
hd_data = pd.read_csv("../data/raw/heart.csv", header=None, names=names)

#in case some data comes in as string
#convert to numeric and coerce errors to NaN
for col in hd_data.columns:  # Iterate over chosen columns
    hd_data[col] = pd.to_numeric(hd_data[col], errors='coerce')
    
#drop nulls
hd_data.dropna(inplace=True)

#non-ohe data---------------------------------
   
#load the X and y set as a numpy array
X_hd = hd_data.drop('target', axis=1).values
y_hd = hd_data['target'].values

#build the train and test sets
X_hd_train, X_hd_test, y_hd_train, y_hd_test = \
    sklearn.model_selection.train_test_split(X_hd, y_hd, random_state=1)

#now do ohe-----------------------------------

#function to do one hot encoding for categorical columns
def create_dummies(data, cols, drop1st=True):
    for c in cols:
        dummies_df = pd.get_dummies(data[c], prefix=c, drop_first=drop1st)  
        data=pd.concat([data, dummies_df], axis=1)
        data = data.drop([c], axis=1)
    return data

cat_cols = ['cp', 'restecg', 'slope', 'ca', 'thal']
hd_data = create_dummies(hd_data, cat_cols)
    
#load the X and y set as a numpy array
X_hd_ohe = hd_data.drop('target', axis=1).values
y_hd_ohe = hd_data['target'].values

#build the train and test sets
X_hd_ohe_train, X_hd_ohe_test, y_hd_ohe_train, y_hd_ohe_test = \
    sklearn.model_selection.train_test_split(X_hd_ohe, y_hd_ohe, \
                                             random_state=1)

#### Function to delete the output and temp directories of auto-sklearn

You'll need to clear the previous folders to avoid overwrite errors. Alternatively, you can create new output directories.

In [7]:
def cleanup(directories_, delete_):
    for d in directories_:
        if delete_:
            print('deleting', d)
            completed = subprocess.run(
                ['rm', '-rf', d],
                stdout=subprocess.PIPE,
            )
            print(completed.stdout.decode('utf-8'))

#### Build a model on ohe data with holdout

In [8]:
%%time

#set and clear the output directories
directories = ['../results/tmp_hd_holdout', \
               '../results/out_hd_holdout']
cleanup(directories, True)

#build the auto-sklearn routine
automl_hd_ohe = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=60,
    per_run_time_limit=30,
    tmp_folder=directories[0],
    output_folder=directories[1],
    disable_evaluator_output=False,
    resampling_strategy='holdout',
    resampling_strategy_arguments={'train_size': 0.67}
)

#call it
automl_hd_ohe.fit(X_hd_ohe_train, y_hd_ohe_train, \
                  dataset_name='heart_disease')

#save the predicitons
predictions_hd_ohe = automl_hd_ohe.predict(X_hd_ohe_test)

deleting ../results/tmp_hd_holdout

deleting ../results/out_hd_holdout

[WARNING] [2020-02-13 15:51:57,048:EnsembleBuilder(1):heart_disease] No models better than random - using Dummy Score!
[WARNING] [2020-02-13 15:51:57,072:EnsembleBuilder(1):heart_disease] No models better than random - using Dummy Score!
CPU times: user 6.58 s, sys: 296 ms, total: 6.88 s
Wall time: 55.6 s


#### Fitting with autosklearn

A common mistake is to call **fit_ensemble()** after already running **fit()**. **fit()** both optimizes the machine learning models and builds an ensemble out of them. To disable ensembling when running **fit()** (with parallel instances for example) set ensemble_size to 0. Then **fit_ensemble()** would be needed once all models have been built.

To save fitted models, use typical [pickle procedures](https://scikit-learn.org/stable/modules/model_persistence.html#persistence-example).

#### Metrics

Accuracy, sprint stats, and model details are available. 

Later we will run auto-sklearn in parallel. Note the number of models built here and compare it to the number built with parallelization turned on. 

The model details give you insight into what auto-sklearn is doing under the hood. You can see the modeling algorithm used and all the parameter settings. 

In [9]:
print('Accuracy:')
print(sklearn.metrics.accuracy_score(y_hd_ohe_test, \
                                     predictions_hd_ohe))
print(' ')
print('-----------------------------------------')
print(' ')
print('Sprint Stats:')
print(automl_hd_ohe.sprint_statistics())
print(' ')
print('-----------------------------------------')
print(' ')
print('Model Details:')
print(automl_hd_ohe.show_models())

Accuracy:
0.7236842105263158
 
-----------------------------------------
 
Sprint Stats:
auto-sklearn results:
  Dataset name: heart_disease
  Metric: accuracy
  Best validation score: 0.880000
  Number of target algorithm runs: 22
  Number of successful target algorithm runs: 21
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

 
-----------------------------------------
 
Model Details:
[(0.860000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'categorical_encoding:__choice__': 'one_hot_encoding', 'classifier:__choice__': 'sgd', 'imputation:strategy': 'most_frequent', 'preprocessor:__choice__': 'nystroem_sampler', 'rescaling:__choice__': 'minmax', 'categorical_encoding:one_hot_encoding:use_minimum_fraction': 'False', 'classifier:sgd:alpha': 5.6261158385241367e-05, 'classifier:sgd:average': 'True', 'classifier:sgd:fit_intercept': 'True', 'classifier:s

#### Do the same thing (build a model on ohe data with holdout) but this time with parallelization turned on

In [10]:
%%time

#set and clear the output directories
directories_parallel = ['../results/tmp_hd_holdout_parallel', \
                        '../results/out_hd_holdout_parallel']
cleanup(directories_parallel, True)

#build the auto-sklearn routine
automl_hd_ohe_p = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=60,
    per_run_time_limit=30,
    tmp_folder=directories_parallel[0],
    output_folder=directories_parallel[1],
    disable_evaluator_output=False,
    resampling_strategy='holdout',
    resampling_strategy_arguments={'train_size': 0.67},
    
    #turn on parallelization
    n_jobs=4,
    seed=5,
    
    delete_output_folder_after_terminate=False,
    delete_tmp_folder_after_terminate=False,
)

#call it
automl_hd_ohe_p.fit(X_hd_ohe_train, y_hd_ohe_train, \
                    dataset_name='heart_disease')

#save the predicitons
predictions_hd_ohe_p = automl_hd_ohe_p.predict(X_hd_ohe_test)

deleting ../results/tmp_hd_holdout_parallel

deleting ../results/out_hd_holdout_parallel

[WARNING] [2020-02-13 16:02:17,977:EnsembleBuilder(236996814):heart_disease] No models better than random - using Dummy Score!
[WARNING] [2020-02-13 16:02:17,984:EnsembleBuilder(236996814):heart_disease] No models better than random - using Dummy Score!
CPU times: user 3.25 s, sys: 320 ms, total: 3.57 s
Wall time: 57.4 s


In [11]:
print('Accuracy:')
print(sklearn.metrics.accuracy_score(y_hd_ohe_test, \
                                     predictions_hd_ohe_p))
print(' ')
print('-----------------------------------------')
print(' ')
print('Sprint Stats:')
print(automl_hd_ohe_p.sprint_statistics())

Accuracy:
0.7763157894736842
 
-----------------------------------------
 
Sprint Stats:
auto-sklearn results:
  Dataset name: heart_disease
  Metric: accuracy
  Best validation score: 0.880000
  Number of target algorithm runs: 43
  Number of successful target algorithm runs: 40
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0



In [12]:
print('Model Details:')
print(automl_hd_ohe_p.show_models())

Model Details:
[(0.220000, SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'categorical_encoding:__choice__': 'no_encoding', 'classifier:__choice__': 'sgd', 'imputation:strategy': 'median', 'preprocessor:__choice__': 'no_preprocessing', 'rescaling:__choice__': 'quantile_transformer', 'classifier:sgd:alpha': 1.0379182676593809e-05, 'classifier:sgd:average': 'False', 'classifier:sgd:fit_intercept': 'True', 'classifier:sgd:learning_rate': 'invscaling', 'classifier:sgd:loss': 'perceptron', 'classifier:sgd:penalty': 'l2', 'classifier:sgd:tol': 0.006867085178762876, 'rescaling:quantile_transformer:n_quantiles': 87, 'rescaling:quantile_transformer:output_distribution': 'normal', 'classifier:sgd:eta0': 2.2316207761131583e-05, 'classifier:sgd:power_t': 0.34160555954890687},
dataset_properties={
  'task': 1,
  'sparse': False,
  'multilabel': False,
  'multiclass': False,
  'target_type': 'classification',
  'signed': False})),
(0.140000, SimpleClassificationPipeline({'balancing

#### Try with feat_type option instead of ohe (still using parallel and holdout)

In [13]:
%%time

#set and clear the output directories
directories_parallel_ft = ['../results/tmp_hd_holdout_parallel_ft', \
                           '../results/out_hd_holdout_parallel_ft']
cleanup(directories_parallel_ft, True)

#build the auto-sklearn routine
automl_hd_ft_p = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=60,
    per_run_time_limit=30,
    tmp_folder=directories_parallel_ft[0],
    output_folder=directories_parallel_ft[1],
    disable_evaluator_output=False,
    # 'holdout' with 'train_size'=0.67 is the default argument setting
    # for AutoSklearnClassifier. It is explicitly specified in this example
    # for demonstrational purpose.
    resampling_strategy='holdout',
    resampling_strategy_arguments={'train_size': 0.67},
    n_jobs=4,
    seed=5,
    delete_output_folder_after_terminate=False,
    delete_tmp_folder_after_terminate=False,
)

feat_type = ['Numerical','Numerical','Categorical','Numerical',\
             'Numerical', 'Numerical','Categorical', 'Numerical',\
             'Numerical','Numerical', 'Categorical','Numerical',\
             'Categorical']

#call it
automl_hd_ft_p.fit(X_hd_train, y_hd_train, \
                   dataset_name='heart_disease', feat_type=feat_type)

#save the predicitons
predictions_hd_ft_p = automl_hd_ft_p.predict(X_hd_test)

deleting ../results/tmp_hd_holdout_parallel_ft

deleting ../results/out_hd_holdout_parallel_ft

[WARNING] [2020-02-13 16:03:15,391:EnsembleBuilder(236996814):heart_disease] No models better than random - using Dummy Score!
[WARNING] [2020-02-13 16:03:15,477:EnsembleBuilder(236996814):heart_disease] No models better than random - using Dummy Score!
CPU times: user 3.72 s, sys: 232 ms, total: 3.96 s
Wall time: 57.4 s


In [14]:
print('Accuracy:')
print(sklearn.metrics.accuracy_score(y_hd_ohe_test, \
                                     predictions_hd_ft_p))
print(' ')
print('-----------------------------------------')
print(' ')
print('Sprint Stats:')
print(automl_hd_ft_p.sprint_statistics())

Accuracy:
0.7236842105263158
 
-----------------------------------------
 
Sprint Stats:
auto-sklearn results:
  Dataset name: heart_disease
  Metric: accuracy
  Best validation score: 0.866667
  Number of target algorithm runs: 50
  Number of successful target algorithm runs: 47
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0



In [15]:
print('Model Details:')
print(automl_hd_ft_p.show_models())

Model Details:
[(0.240000, SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'categorical_encoding:__choice__': 'one_hot_encoding', 'classifier:__choice__': 'multinomial_nb', 'imputation:strategy': 'mean', 'preprocessor:__choice__': 'liblinear_svc_preprocessor', 'rescaling:__choice__': 'minmax', 'categorical_encoding:one_hot_encoding:use_minimum_fraction': 'False', 'classifier:multinomial_nb:alpha': 92.44256225709728, 'classifier:multinomial_nb:fit_prior': 'False', 'preprocessor:liblinear_svc_preprocessor:C': 11433.184251681738, 'preprocessor:liblinear_svc_preprocessor:dual': 'False', 'preprocessor:liblinear_svc_preprocessor:fit_intercept': 'True', 'preprocessor:liblinear_svc_preprocessor:intercept_scaling': 1, 'preprocessor:liblinear_svc_preprocessor:loss': 'squared_hinge', 'preprocessor:liblinear_svc_preprocessor:multi_class': 'ovr', 'preprocessor:liblinear_svc_preprocessor:penalty': 'l1', 'preprocessor:liblinear_svc_preprocessor:tol': 0.00014616338472666772},
dataset_

#### Try with CV instead of Holdout (using ohe and parallel)

CV requires an extra step to fit our ensemble on all the data. During **fit()**, models are fit on individual cross-validation folds. To use all available data, we call **refit()** which trains all models in the final ensemble on the whole dataset. Also, when using CV, **fit()** changes the data in place, but refit needs the original data. So we use the **copy()** function. In practice, you might want to reload the data.

In [16]:
# %%time

# #set and clear the output directories
# directories_parallel_cv = ['../results/tmp_hd_holdout_parallel_cv', \
#                            '../results/out_hd_holdout_parallel_cv']
# cleanup(directories_parallel_cv, True)

# #build the auto-sklearn routine
# automl_hd_cv_p = autosklearn.classification.AutoSklearnClassifier(
#     time_left_for_this_task=60,
#     per_run_time_limit=30,
#     tmp_folder=directories_parallel_cv[0],
#     output_folder=directories_parallel_cv[1],
#     disable_evaluator_output=False,
#     # 'holdout' with 'train_size'=0.67 is the default argument setting
#     # for AutoSklearnClassifier. It is explicitly specified in this example
#     # for demonstrational purpose.
#     resampling_strategy='cv',
#     resampling_strategy_arguments={'folds': 5},
#     n_jobs=4,
#     seed=5,
#     delete_output_folder_after_terminate=False,
#     delete_tmp_folder_after_terminate=False,
# )

# #call it
# automl_hd_cv_p.fit(X_hd_ohe_train.copy(), y_hd_ohe_train.copy(), \
#                    dataset_name='heart_disease')
# automl_hd_cv_p.refit(X_hd_ohe_train.copy(), y_hd_ohe_train.copy())

# #save the predicitons
# predictions_hd_cv_p = automl_hd_cv_p.predict(X_hd_ohe_test)

In [17]:
# print('Accuracy:')
# print(sklearn.metrics.accuracy_score(y_hd_ohe_test, \
#                                      predictions_hd_cv_p))
# print(' ')
# print('-----------------------------------------')
# print(' ')
# print('Sprint Stats:')
# print(automl_hd_cv_p.sprint_statistics())

In [18]:
# print('Model Details:')
# print(automl_hd_cv_p.show_models())

## Breast Cancer

#### Load the breast cancer data

In [19]:
from sklearn.datasets import load_breast_cancer

'''
Attribute Information:

1) ID number 
2) Diagnosis (M = malignant, B = benign) 
3-32) 

Ten real-valued features are computed for each cell nucleus: 

a) radius (mean of distances from center to points on the perimeter) 
b) texture (standard deviation of gray-scale values) 
c) perimeter 
d) area 
e) smoothness (local variation in radius lengths) 
f) compactness (perimeter^2 / area - 1.0) 
g) concavity (severity of concave portions of the contour) 
h) concave points (number of concave portions of the contour) 
i) symmetry 
j) fractal dimension ("coastline approximation" - 1)
'''

#load from sklearn
X_bc, y_bc = sklearn.datasets.load_breast_cancer(return_X_y=True)

#build the train and test sets
X_bc_train, X_bc_test, y_bc_train, y_bc_test = \
    sklearn.model_selection.train_test_split(X_bc, y_bc, random_state=1)

#### Build a model using holdout and parallelization

In [20]:
%%time

#set and clear the output directorie
directories_bc = ['../results/tmp_bc', '../results/out_bc']
cleanup(directories_bc, True)

#build the auto-sklearn routine
automl_bc = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=60,
    per_run_time_limit=30,
    tmp_folder=directories_bc[0],
    output_folder=directories_bc[1],
    disable_evaluator_output=False,
    # 'holdout' with 'train_size'=0.67 is the default argument setting
    # for AutoSklearnClassifier. It is explicitly specified in this example
    # for demonstrational purpose.
    resampling_strategy='holdout',
    resampling_strategy_arguments={'train_size': 0.67},
    n_jobs=4,
    seed=5,
    delete_output_folder_after_terminate=False,
    delete_tmp_folder_after_terminate=False,
)

#call it
automl_bc.fit(X_bc_train, y_bc_train, dataset_name='breast_cancer')

#save the predicitons
predictions_bc = automl_bc.predict(X_bc_test)

deleting ../results/tmp_bc

deleting ../results/out_bc

[WARNING] [2020-02-13 16:04:13,480:EnsembleBuilder(236996814):breast_cancer] No models better than random - using Dummy Score!
[WARNING] [2020-02-13 16:04:13,574:EnsembleBuilder(236996814):breast_cancer] No models better than random - using Dummy Score!
CPU times: user 2.92 s, sys: 336 ms, total: 3.26 s
Wall time: 56.4 s


In [21]:
print('Accuracy:')
print(sklearn.metrics.accuracy_score(y_bc_test, \
                                     predictions_bc))
print(' ')
print('-----------------------------------------')
print(' ')
print('Sprint Stats:')
print(automl_bc.sprint_statistics())

Accuracy:
0.958041958041958
 
-----------------------------------------
 
Sprint Stats:
auto-sklearn results:
  Dataset name: breast_cancer
  Metric: accuracy
  Best validation score: 0.985816
  Number of target algorithm runs: 38
  Number of successful target algorithm runs: 34
  Number of crashed target algorithm runs: 1
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0



In [22]:
print('Model Details:')
print(automl_bc.show_models())

Model Details:
[(0.640000, SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'categorical_encoding:__choice__': 'one_hot_encoding', 'classifier:__choice__': 'sgd', 'imputation:strategy': 'mean', 'preprocessor:__choice__': 'nystroem_sampler', 'rescaling:__choice__': 'normalize', 'categorical_encoding:one_hot_encoding:use_minimum_fraction': 'True', 'classifier:sgd:alpha': 5.871626569371367e-07, 'classifier:sgd:average': 'False', 'classifier:sgd:fit_intercept': 'True', 'classifier:sgd:learning_rate': 'invscaling', 'classifier:sgd:loss': 'modified_huber', 'classifier:sgd:penalty': 'elasticnet', 'classifier:sgd:tol': 0.019859199949717275, 'preprocessor:nystroem_sampler:kernel': 'poly', 'preprocessor:nystroem_sampler:n_components': 153, 'categorical_encoding:one_hot_encoding:minimum_fraction': 0.0002167910828134983, 'classifier:sgd:epsilon': 1.613923621745244e-05, 'classifier:sgd:eta0': 8.657009255576157e-07, 'classifier:sgd:l1_ratio': 0.0037033148753308357, 'classifier:sgd:po

## Model Run and Accuracy Stats

All in one place for easier comparison.

In [23]:
print("-----------Heart Disease---------------")
print(' ')
print(' ')

print("Model stats HD Holdout:")
print(automl_hd_ohe.sprint_statistics())
print(' ')
print("Accuracy score HD Holdout:")
print(sklearn.metrics.accuracy_score(y_hd_ohe_test, \
                                     predictions_hd_ohe))

print(' ')
print('-----------------------------------------')
print(' ')

print("Model stats HD Holdout Parallel:")
print(automl_hd_ohe_p.sprint_statistics())
print("Accuracy score HD Holdout Parallel:")
print(sklearn.metrics.accuracy_score(y_hd_ohe_test, \
                                     predictions_hd_ohe_p))

print(' ')
print('-----------------------------------------')
print(' ')

#holdout parallel feat_type
print("Model stats HD Holdout Feature Type Parallel:")
print(automl_hd_ft_p.sprint_statistics())
print(' ')
print("Accuracy score HD Holdout Feature Type Parallel:")
print(sklearn.metrics.accuracy_score(y_hd_test, \
                                     predictions_hd_ft_p))

# print(' ')
# print('-----------------------------------------')
# print(' ')

# #cross validation parallel
# print("Model stats HD CV Parllel:")
# print(automl_hd_cv_p.sprint_statistics())
# print(' ')
# print("Accuracy score HD CV Parallel:")
# print(sklearn.metrics.accuracy_score(y_hd_ohe_test, \
#                                      predictions_hd_cv_p))

print(' ')
print(' ')

print("-----------Breast Cancer---------------")
print(' ')
print(' ')

print("Model stats BC Holdout Parallel:")
print(automl_bc.sprint_statistics())
print(' ')
print("Accuracy score BC Holdout Parallel:")
print(sklearn.metrics.accuracy_score(y_bc_test, \
                                     predictions_bc))

-----------Heart Disease---------------
 
 
Model stats HD Holdout:
auto-sklearn results:
  Dataset name: heart_disease
  Metric: accuracy
  Best validation score: 0.880000
  Number of target algorithm runs: 22
  Number of successful target algorithm runs: 21
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

 
Accuracy score HD Holdout:
0.7236842105263158
 
-----------------------------------------
 
Model stats HD Holdout Parallel:
auto-sklearn results:
  Dataset name: heart_disease
  Metric: accuracy
  Best validation score: 0.880000
  Number of target algorithm runs: 43
  Number of successful target algorithm runs: 40
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

Accuracy score HD Holdout Parallel:
0.7763157894736842
 
--------------------

## Save to Domino Stats File

To keep things simple, we pick one of the hd models. Saving stats to this file [allows Domino to track and trend them in the Experiment Manager](https://support.dominodatalab.com/hc/en-us/articles/204348169-Diagnostic-statistics-with-dominostats-json) when this notebook is run as a batch or scheduled job.

In [24]:
hd_acc = sklearn.metrics.accuracy_score(y_hd_ohe_test, \
                                        predictions_hd_ohe_p)
bc_acc = sklearn.metrics.accuracy_score(y_bc_test, \
                                        predictions_bc)

import json
with open('../dominostats.json', 'w') as f:
    f.write(json.dumps( {"HD_ACC": hd_acc, "BC_ACC": bc_acc}))